In [5]:
import numpy as np

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

class NeuralNetwork:
    def __init__(self, x, y, learning_rate=0.01):
        self.input      = np.array(x) / np.max(x)  # Normalize inputs
        self.weights1   = np.random.rand(self.input.shape[1], 6)  # Increased hidden layer neurons
        self.weights2   = np.random.rand(6, 1)      
        self.y          = np.array(y).reshape(-1, 1) / np.max(y)  # Normalize output
        self.output     = np.zeros(self.y.shape)
        self.learning_rate = learning_rate
    
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))
 
    def backprop(self):
        # Application of the chain rule to find the derivative of the loss function with respect to weights2 and weights1
        d_weights2 = np.dot(self.layer1.T, 
                            (2 * (self.y - self.output) * sigmoid_derivative(self.output)))
        d_weights1 = np.dot(self.input.T,
                            (np.dot(2 * (self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) 
                            * sigmoid_derivative(self.layer1)))
 
        # Update the weights with the learning rate
        self.weights1 += self.learning_rate * d_weights1
        self.weights2 += self.learning_rate * d_weights2

    def train(self, epochs):
        for epoch in range(epochs):
            self.feedforward()
            self.backprop()
            if epoch % 1000 == 0:
                # Mean Squared Error Loss
                loss = np.mean(np.square(self.y - self.output))
                print(f"Epoch {epoch} Loss: {loss}")

inputs = [[1, 2, 3],
          [2, 4, 6],
          [3, 6, 9],
          [4, 8, 12],
          [5, 10, 15]]
outputs = [0, 1, 0, 1, 0]

# Initialize the neural network with a learning rate
app_1 = NeuralNetwork(inputs, outputs, learning_rate=0.01)

print("initial weights1:\n", app_1.weights1)
print("initial weights2:\n", app_1.weights2)


# Train the network for 20,000 epochs
app_1.train(epochs=20000)

# Display the final weights and output
print("Final weights1:\n", app_1.weights1)
print("Final weights2:\n", app_1.weights2)
print("Final output after training:\n", app_1.output * np.max(outputs))  # Denormalize output
print("Expected output (y):\n", app_1.y * np.max(outputs))  # Denormalize expected output

initial weights1:
 [[0.57655593 0.51408891 0.65499847 0.00971623 0.59802817 0.13426049]
 [0.50328864 0.89486999 0.81969989 0.35925765 0.97410386 0.53116146]
 [0.43812297 0.38843755 0.34943935 0.1443839  0.70404024 0.22287655]]
initial weights2:
 [[0.01471724]
 [0.84623049]
 [0.32993113]
 [0.68550576]
 [0.20658774]
 [0.1158958 ]]
Epoch 0 Loss: 0.3989000627305253
Epoch 1000 Loss: 0.2404367162567514
Epoch 2000 Loss: 0.2403970672970047
Epoch 3000 Loss: 0.2403592418500689
Epoch 4000 Loss: 0.24032302799496322
Epoch 5000 Loss: 0.2402882411564809
Epoch 6000 Loss: 0.24025472063934106
Epoch 7000 Loss: 0.24022232664558768
Epoch 8000 Loss: 0.24019093772927436
Epoch 9000 Loss: 0.24016044860867342
Epoch 10000 Loss: 0.24013076827155144
Epoch 11000 Loss: 0.24010181832139352
Epoch 12000 Loss: 0.2400735315224976
Epoch 13000 Loss: 0.24004585051007696
Epoch 14000 Loss: 0.24001872663829116
Epoch 15000 Loss: 0.23999211894472908
Epoch 16000 Loss: 0.239965993214512
Epoch 17000 Loss: 0.23994032113100902
Epoch 

In [ ]:
class NeuralNetwork2:
    def __init__(self, x, y):
        self.input      = np.array(x)
        self.weights1   = np.random.rand(self.input.shape[1],4) 
        # the 4 specifies the number of neurons in the hidden layer. Here's how it breaks down:
        self.weights2   = np.random.rand(4,1)                 
        self.y          = np.array(y)
        self.output     = np.zeros(self.y.shape)

app_1_inputs = [[1,2,3],
                [4,5,6],
                [7,8,9]]
app_1_outputs= [1,5,10]

app_1 = NeuralNetwork2(app_1_inputs, app_1_outputs)

app_1.weights1, app_1.output, app_1.y

(array([[0.16387844, 0.44053379, 0.67081371, 0.08014979],
        [0.26691855, 0.56196013, 0.92606066, 0.22048417],
        [0.72562238, 0.56651683, 0.20371703, 0.67436192]]),
 array([0., 0., 0.]),
 array([ 1,  5, 10]))

In [20]:
import numpy as np

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

class Node:
    def __init__(self, value=0, operation=None, inputs=None):
        self.value = value  # Output of the node
        self.grad = 0  # Gradient of the node, initialized to 0
        self.operation = operation  # The operation that this node performs
        self.inputs = inputs if inputs else []  # Inputs to this node (list of nodes)
        self.output_nodes = []  # Nodes that depend on this one for their calculation
        
        # If the operation is set, we connect the input nodes to this one
        if self.inputs:
            for input_node in self.inputs:
                input_node.output_nodes.append(self)

    def forward(self):
        """ Compute the output of the node (forward pass). """
        if self.operation:
            self.value = self.operation(*[input.value for input in self.inputs])
        return self.value

    def backward(self):
        """ Compute the gradient of the node (backward pass). """
        if self.operation:
            # Get the derivative of the operation with respect to its inputs
            self.grad = self.operation.gradient(*[input.value for input in self.inputs])
        
        # Propagate the gradient to the input nodes
        for input_node in self.inputs:
            input_node.grad += self.grad * input_node.value  # Chain rule

class Add:
    @staticmethod
    def __call__(self, x, y):
        return x + y

    @staticmethod
    def gradient(x, y):
        return 1, 1  # Gradient of x + y with respect to x and y

class Multiply:
    @staticmethod
    def __call__(self, x, y):
        return x * y

    @staticmethod
    def gradient(x, y):
        return y, x  # Gradient of x * y with respect to x and y

class ComputationalGraph:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        """ Add a node to the computational graph """
        self.nodes.append(node)

    def forward(self):
        """ Perform a forward pass on all nodes """
        for node in self.nodes:
            node.forward()

    def backward(self):
        """ Perform a backward pass to compute gradients """
        # Initialize a dictionary to hold input gradients
        input_gradients = {}

        for node in reversed(self.nodes):
            node.backward()

            # Store the gradients of the input nodes in the dictionary
            for input_node in node.inputs:
                if input_node not in input_gradients:
                    input_gradients[input_node] = input_node.grad
                else:
                    input_gradients[input_node] += input_node.grad
        
        return input_gradients

class NeuralNetwork:
    def __init__(self, x, y, learning_rate=0.01):
        self.input = np.array(x) / np.max(x)  # Normalize inputs
        self.weights1 = np.random.rand(self.input.shape[1], 6)  # Increased hidden layer neurons
        self.weights2 = np.random.rand(6, 1)      
        self.y = np.array(y).reshape(-1, 1) / np.max(y)  # Normalize output
        self.output = np.zeros(self.y.shape)
        self.learning_rate = learning_rate
        
    def feedforward(self):
        """ Forward pass to calculate the output """
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))  # Hidden layer output
        self.output = sigmoid(np.dot(self.layer1, self.weights2))  # Final output

        # Threshold output to get binary 0 or 1
        self.output = np.where(self.output > 0.5, 1, 0)

    def backprop(self):
        """ Backward pass using the chain rule """
        # Compute error
        output_error = self.y - self.output
        output_delta = output_error * sigmoid_derivative(self.output)
        
        # Compute gradients for weights2
        d_weights2 = np.dot(self.layer1.T, output_delta)
        
        # Compute error for hidden layer
        hidden_error = output_delta.dot(self.weights2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.layer1)
        
        # Compute gradients for weights1
        d_weights1 = np.dot(self.input.T, hidden_delta)
        
        # Update weights
        self.weights1 += self.learning_rate * d_weights1
        self.weights2 += self.learning_rate * d_weights2

    def train(self, epochs):
        for epoch in range(epochs):
            self.feedforward()
            self.backprop()
            if epoch % 1000 == 0:
                # Mean Squared Error Loss
                loss = np.mean(np.square(self.y - self.output))
                print(f"Epoch {epoch} Loss: {loss}")

# Define input and expected output
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])

# Create neural network
nn = NeuralNetwork(x, y, learning_rate=0.001)

# Train the neural network for 10000 epochs
nn.train(epochs=10000)

# After training, output the final weights and predictions
print("Final weights1:\n", nn.weights1)
print("Final weights2:\n", nn.weights2)
print("Final output after training:\n", nn.output)
print("Expected output (y):\n", y)


Epoch 0 Loss: 0.5
Epoch 1000 Loss: 0.5
Epoch 2000 Loss: 0.5
Epoch 3000 Loss: 0.75
Epoch 4000 Loss: 0.75
Epoch 5000 Loss: 0.75
Epoch 6000 Loss: 0.5
Epoch 7000 Loss: 0.25
Epoch 8000 Loss: 0.5
Epoch 9000 Loss: 0.25
Final weights1:
 [[ 4.92701620e-01  4.90674953e-01  3.43392488e-01  8.48144441e-01
   2.96834945e-01  7.52747765e-01]
 [ 1.41610696e-01  5.74480045e-01  5.72225035e-01 -1.11810118e-04
   2.87419395e-01  3.81680693e-01]]
Final weights2:
 [[-0.0064345 ]
 [-0.48157486]
 [ 0.40523243]
 [ 0.11921238]
 [ 0.12236263]
 [-0.12846414]]
Final output after training:
 [[1]
 [0]
 [0]
 [0]]
Expected output (y):
 [0 1 1 0]
